In [14]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential , save_model, load_model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Dropout, SpatialDropout2D, BatchNormalization, Input,Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [7]:
img_height, img_width = 128, 128
batch_size=32
# Data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input,  # Apply ResNet preprocessing
    rescale=1./255, 
    rotation_range=45,
    width_shift_range=0.2, 
    height_shift_range=0.2,  
    shear_range=0.2, 
    zoom_range=0.3,  
    horizontal_flip=True,
    vertical_flip= True ,
    fill_mode='nearest' 
)
# No Augmentation applied for both Validation and Testing Datasets
val_test_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Training',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # 'categorical' for multi-class classification, one-hot encoded vectors
)

val_generator = val_test_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    'D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Dataset/Teeth_Dataset/Testing',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [17]:
test_step=test_generator.n//test_generator.batch_size


In [8]:
vgg16_pretrained = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# # Freeze the layers you don't want to train
# for layer in vgg16_pretrained.layers:
#     layer.trainable = False

#Alternatively, you can selectively unfreeze some layers like this:
for idx, layer in enumerate(vgg16_pretrained.layers):
    print(f"Index: {idx}, Layer Name: {layer.name} , Layer Type: {layer.__class__.__name__}")


Index: 0, Layer Name: input_2 , Layer Type: InputLayer
Index: 1, Layer Name: block1_conv1 , Layer Type: Conv2D
Index: 2, Layer Name: block1_conv2 , Layer Type: Conv2D
Index: 3, Layer Name: block1_pool , Layer Type: MaxPooling2D
Index: 4, Layer Name: block2_conv1 , Layer Type: Conv2D
Index: 5, Layer Name: block2_conv2 , Layer Type: Conv2D
Index: 6, Layer Name: block2_pool , Layer Type: MaxPooling2D
Index: 7, Layer Name: block3_conv1 , Layer Type: Conv2D
Index: 8, Layer Name: block3_conv2 , Layer Type: Conv2D
Index: 9, Layer Name: block3_conv3 , Layer Type: Conv2D
Index: 10, Layer Name: block3_pool , Layer Type: MaxPooling2D
Index: 11, Layer Name: block4_conv1 , Layer Type: Conv2D
Index: 12, Layer Name: block4_conv2 , Layer Type: Conv2D
Index: 13, Layer Name: block4_conv3 , Layer Type: Conv2D
Index: 14, Layer Name: block4_pool , Layer Type: MaxPooling2D
Index: 15, Layer Name: block5_conv1 , Layer Type: Conv2D
Index: 16, Layer Name: block5_conv2 , Layer Type: Conv2D
Index: 17, Layer Name:

In [9]:
# # Freeze the layers you don't want to train
# for layer in vgg16_pretrained.layers:
#     layer.trainable = False

# Alternatively, you can selectively unfreeze some layers like this:
for layer in vgg16_pretrained.layers[:15]:
    layer.trainable = False
for layer in vgg16_pretrained.layers[15:]:
    layer.trainable = True

In [10]:
# Build your custom model architecture
model = Sequential()

# Add the pre-trained VGG16 base model
model.add(vgg16_pretrained)

# Add custom layers on top of the VGG16 base model
model.add(Flatten())  # Flatten the output from the VGG16 model
model.add(Dense(512, activation='relu'))  # Add a fully connected layer
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(256, activation='relu'))  # Add another fully connected layer
model.add(Dropout(0.5))  # Add another dropout layer
model.add(Dense(7, activation='softmax'))  # Final output layer with softmax (assuming 7 classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 7)                 1

In [11]:
# Defining early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 0, 
    restore_best_weights = True
)

CheckPoint = tf.keras.callbacks.ModelCheckpoint(
    'VGG16.h5' , save_best_only=True
)

history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
     callbacks=[early_stopping , CheckPoint]
)

Epoch 1/100
97/97 [==============================] - 22s 199ms/step - loss: 1.9007 - accuracy: 0.2433 - val_loss: 1.4803 - val_accuracy: 0.4553
Epoch 2/100
97/97 [==============================] - 17s 171ms/step - loss: 1.5851 - accuracy: 0.3777 - val_loss: 1.3276 - val_accuracy: 0.4796
Epoch 3/100
97/97 [==============================] - 17s 171ms/step - loss: 1.3909 - accuracy: 0.4717 - val_loss: 1.0806 - val_accuracy: 0.6459
Epoch 4/100
97/97 [==============================] - 17s 172ms/step - loss: 1.2279 - accuracy: 0.5394 - val_loss: 0.8362 - val_accuracy: 0.7189
Epoch 5/100
97/97 [==============================] - 17s 172ms/step - loss: 1.0688 - accuracy: 0.6041 - val_loss: 0.7198 - val_accuracy: 0.7442
Epoch 6/100
97/97 [==============================] - 16s 169ms/step - loss: 1.0088 - accuracy: 0.6359 - val_loss: 0.8798 - val_accuracy: 0.6887
Epoch 7/100
97/97 [==============================] - 17s 170ms/step - loss: 0.8542 - accuracy: 0.7013 - val_loss: 0.6452 - val_accuracy:

In [12]:
save_model = model.save('D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Model/VGG16.h5')

In [15]:
new_model = load_model('D:/Mohamed Sheriff/Projects/Computer Vision Internship - Cellula Technologies/Teeth Classification/Model/VGG16.h5')

In [18]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = new_model.evaluate(test_generator, steps=test_step)
print("Test accuracy:", test_accuracy)

32/32 [==============================] - 11s 348ms/step - loss: 0.0236 - accuracy: 0.9893
Test accuracy: 0.9892578125
